# Fine-tune Gemma-2-2b (Self-Healing Edition)

**Instructions:**
1. Click **`Run All`**.
2. The first cell will auto-restart the kernel once.
3. Training will start automatically after the self-restart.


In [ ]:
# MAGIC CELL: Install -> Auto-Restart -> Load
import os, sys, subprocess, time

def is_env_ready():
    try:
        import unsloth, torch, torchvision
        from transformers.processing_utils import Unpack
        return True
    except:
        return False

if not is_env_ready():
    print('🚀 Step 1: Installing stable environment...')
    subprocess.run('pip uninstall -y torchao unsloth unsloth_zoo transformers -q', shell=True)
    
    pkgs = [
        'torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121',
        'fsspec==2024.9.0 datasets==4.2.0',
        'transformers>=4.45.0 peft accelerate bitsandbytes trl',
        '"unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"',
        'xformers==0.0.28.post3'
    ]
    
    for pkg in pkgs:
        print(f'   Installing {pkg.split(" ")[0]}...')
        subprocess.run(f'pip install -q --no-cache-dir {pkg}', shell=True)

    print('\n✅ Setup complete! Auto-restarting...')
    os._exit(0)

import unsloth, torch
from unsloth import FastLanguageModel
print('✅ Environment Ready!')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/gemma-2-2b-it-bnb-4bit',
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
print('✅ Model Loaded!')


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
model.print_trainable_parameters()


In [ ]:
from datasets import load_dataset
import glob
for p in ["/kaggle/input/audit-dataset/audit_dataset.jsonl", "/kaggle/input/*/audit_dataset.jsonl"]:
    m = glob.glob(p)
    if m: dataset_path = m[0]; break
else: dataset_path = "audit_dataset.jsonl"

dataset = load_dataset("json", data_files=dataset_path, split="train")
ALPACA = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}\n\n### Response:
{}"""
dataset = dataset.map(lambda x: {
    "text": [ALPACA.format(i, o) + tokenizer.eos_token for i, o in zip(x['instruction'], x['output'])]
}, batched=True)
print(f'✅ Loaded {len(dataset)} examples')


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        report_to="none",
    ),
)
trainer.train()


In [ ]:
model.save_pretrained("audit-gemma-v1")
tokenizer.save_pretrained("audit-gemma-v1")
print("✅ Saved to audit-gemma-v1/")
